# GenerateFramesFiles.ipynb

### Notebook for generating frames files for LIGO data analysis, whether from NR simulation .h5 files, or using the a gravitational waveform surrogate model

Maria Okounkova (mokounkova@flatironinstitute.org)

### Imports and setup

Note that the surrogate model methods will generate deprecation warnings

In [1]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import h5py
from astropy import constants as const
from scipy.interpolate import InterpolatedUnivariateSpline
from pycbc.detector import Detector
from gwpy.timeseries import TimeSeries
from gwpy.detector import Channel
import os
from shutil import copyfile
import fileinput
import seaborn as sns
import math
from math import pi
import pycbc
from pycbc.filter.matchedfilter import overlap
import json
from ipynb.fs.full.Auxiliary import ReadExtrapolatedModes, EvaluateSurrogate, SubtractPeakTime, dt_eval, df_eval, \
                                    PadAndProject, EllinKm, ComputeSNR, ComputeMultiDetectorSNR, TargetSNR
import gwsurrogate
## if you need the surrogate data: 
## gwsurrogate.catalog.pull('NRSur7dq4')

matplotlib.rcParams['mathtext.fontset'] = 'stix'
matplotlib.rcParams['font.family'] = 'STIXGeneral'
matplotlib.rcParams['axes.labelsize'] = 30
matplotlib.rcParams['xtick.labelsize'] = 24
matplotlib.rcParams['ytick.labelsize'] = 24
matplotlib.rcParams['xtick.major.size'] = 10
matplotlib.rcParams['ytick.major.size'] = 10
matplotlib.rcParams['xtick.top'] = True
matplotlib.rcParams['xtick.direction'] = 'in'
matplotlib.rcParams['xtick.minor.visible'] = True
matplotlib.rcParams['xtick.minor.size'] = 5
matplotlib.rcParams['ytick.minor.size'] = 5
matplotlib.rcParams['legend.fontsize'] = 24
matplotlib.rcParams['legend.frameon'] = True
matplotlib.rcParams['lines.linewidth'] = 1

sur = gwsurrogate.LoadSurrogate('NRSur7dq4')

/home/maria.okounkova/.local/lib/python3.6/site-packages/gwtools/rotations.py:63: UserWarning: Could not import GWFrames, needed for rotations module
  _warnings.warn("Could not import GWFrames, needed for rotations module")
/home/maria.okounkova/.local/lib/python3.6/site-packages/gwtools/__init__.py:11: UserWarning: Could not import rotations, decompositions, or fitfuncs. These are not needed by GWSurrogate.
  _warnings.warn("Could not import rotations, decompositions, or fitfuncs. These are not needed by GWSurrogate.")


setting __package__ to gwsurrogate.new so relative imports work
__name__ = gwsurrogate.new.spline_evaluation
__package__= gwsurrogate.new
setting __package__ to gwsurrogate.new so relative imports work
setting __package__ to gwsurrogate.new so relative imports work


/home/maria.okounkova/.local/lib/python3.6/site-packages/gwsurrogate/new/precessing_surrogate.py:245: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  self.t = h5file['t_ds'].value
/home/maria.okounkova/.local/lib/python3.6/site-packages/gwsurrogate/new/precessing_surrogate.py:272: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  'coefs': group['%s_coefs'%(key)].value,
/home/maria.okounkova/.local/lib/python3.6/site-packages/gwsurrogate/new/precessing_surrogate.py:273: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  'bfOrders': group['%s_bfOrders'%(key)].value
/home/maria.okounkova/.local/lib/python3.6/site-packages/gwsurrogate/new/precessing_surrogate.py:282: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  'coefs': group['%s_%d_coefs'%(key, i)].value,
/home/maria.okounkova/.local/lib/python3.6/site-packages/gwsurrogate/new/precessing_sur

Loaded NRSur7dq4 model


### Helper functions for creating and reading injected parameter dictionaries

In [2]:
def FullParamsDictionary(params_dict):
    """ Given a total mass, q > 1, and spins a_1, a_2, in params_dict, 
        compute m_1, m_2, m_chirp, and chi_eff and add to params_dict
         """
    mass = params_dict['mass']
    q = params_dict['q']
    a_1 = params_dict['a_1']
    a_2 = params_dict['a_2']
    
    m_2 = mass / (1 + q)
    m_1 = q*m_2
    m_chirp = (q/(1+q)**2)**(3/5)*mass
    
    params_dict['m_1'] = m_1
    params_dict['m_2'] = m_2
    params_dict['mass_ratio'] = 1.0/q
    params_dict['chirp_mass'] = m_chirp
    
    ## Assuming aligned spin for now 
    chi_eff = (a_1[2]/m_1 + a_2[2]/m_2)/mass
    params_dict['chi_eff'] = chi_eff
    
    return params_dict

def GetInjectedParameters(p):
    """ Return a dictionary of values from the parameters.json file 
        corresponding to the frames, where p is the path to the file """
    
    f = open(p + '/parameters.json')
    params = json.load(f)
    f.close()
    return params

### Frames files writing method

In [3]:
def GenerateFrames(name, timeH, strainH, timeL, strainL, timeV, strainV, params_dict):
    """ Generate frames files given hanford and livingston data.
        rh_dir is the directory base directory for the data 
        Also make bayeswave files if necessary """

    frames_dir = 'BilbyPE/' + name + '/Frames/'
    bilby_dir = 'BilbyPE/' + name + '/'
    
    os.makedirs(frames_dir)
    
    dt = dt_eval(timeH)

    H1 = TimeSeries(strainH,sample_rate=1/dt,epoch=timeH[0],channel="H1:LDAS_STRAIN")
    L1 = TimeSeries(strainL,sample_rate=1/dt,epoch=timeL[0],channel="L1:LDAS_STRAIN")
    V1 = TimeSeries(strainV,sample_rate=1/dt,epoch=timeL[0],channel="V1:LDAS_STRAIN")

    H1.write(frames_dir + "H1.gwf")
    L1.write(frames_dir + "L1.gwf")
    V1.write(frames_dir + "V1.gwf")
    
    ## Copy over the bilby files and replace path variables
    for file in ['aLIGOZeroDetHighPower-PSD_25Hz.txt', 'Frame.prior', 'Frames.py', 
                 'run_bilby.dag', 'run_bilby.sh', 'run_bilby.sub']:
        
        ## Copy over base file
        copyfile('BilbyPE/Files/' + file, bilby_dir + file)
        
        ## Replace path variables
        with fileinput.FileInput(bilby_dir + file, inplace=True) as f:
            for line in f:
                print(line.replace('FRAMES_DIR', name), end='')
            
    ## Logs directory for Bilby
    os.mkdir(bilby_dir + 'Logs')
    
    ## Bayeswave files
    bw_dir = bilby_dir + 'BayesWave'
    os.mkdir(bw_dir)
    
    # Make the bayeswave cache files
    start_dir = '/home/maria.okounkova/BeyondGRAnalysis/'
    with open(bw_dir + '/H1_Cache.lcf', 'a') as file:
        file.write("-\t-\t-\t-\tfile://localhost" + start_dir + frames_dir + 'H1.gwf' + '\n')
    with open(bw_dir + '/L1_Cache.lcf', 'a') as file:
        file.write("-\t-\t-\t-\tfile://localhost" + start_dir + frames_dir + 'L1.gwf' + '\n')
    with open(bw_dir + '/V1_Cache.lcf', 'a') as file:
        file.write("-\t-\t-\t-\tfile://localhost" + start_dir + frames_dir + 'V1.gwf' + '\n')

    # Copy over the bayewave ini files
    copyfile('run_bw.sh', bw_dir + '/run_bw.sh')
    copyfile('dCS.ini', bw_dir + '/dCS.ini')
    
    # Change the path to the cache files in the copied dCS.ini file
    with fileinput.FileInput(bw_dir + '/dCS.ini', inplace=True) as file:
        for line in file:
            print(line.replace('CACHE_DIR', bw_dir), end='')

### Create the frames files - evaluate the waveforms, project the data, and write frames files

In [4]:
def EvaluateNRWaveform(p_in, p_out, params_dict, desired_snr, ell = "0p0"):
    
    """
    Given a directory p_out in which the numerical relativity waveform lives (in an SXS format), 
    apply the astrophysical parameters given in params_dict to the waveform, and write frames
    to directory p_out. 
    
    desired_snr corresponds to the SNR we would like for the frames files, which is 
    achieved by adjusting the distance. 
    """
    time, h_plus, h_cross, amp = ReadExtrapolatedModes(p_in, params_dict, interpolate = False)
    timeH, strainH, timeL, strainL, timeV, strainV = PadAndProject(time, h_plus, h_cross, params_dict, verbose = True)
    
    ## Update distance for target SNR
    strainH, strainL, strainV, params_dict = TargetSNR(timeH, strainH, timeL, strainL, timeV, strainV, params_dict, desired_snr)
    GenerateFrames(p_out, timeH, strainH, timeL, strainL, timeV, strainV, params_dict)
    
    ## Compute SNRs
    SNR_H = ComputeSNR(timeH, strainH)
    SNR_L = ComputeSNR(timeL, strainL)
    SNR_V = ComputeSNR(timeV, strainV)
    SNR = ComputeMultiDetectorSNR(timeH, strainH, timeL, strainL, timeV, strainV)
    print("Network SNR: ", SNR)
    
    ## Print out the parameter values
    params_dict = FullParamsDictionary(params_dict)
    params_dict['source'] = p_in
    params_dict['H_SNR'] = SNR_H
    params_dict['L_SNR'] = SNR_L
    params_dict['V_SNR'] = SNR_V
    params_dict['SNR'] = SNR
    params_dict['ell'] = ell 
    params_dict['ell_km'] = EllinKm(ell, params_dict['mass'])
    with open('BilbyPE/' + p_out + '/parameters.json', 'w') as fp:
        json.dump(params_dict, fp)
    fp.close()

def EvaluateSurrogateWaveform(p_out, params_dict, desired_snr):
    """
    Given a directory dictionary of BBH parameters params_dict, evaluate the surrogate model 
    and write frames to directory p_out. 
    
    desired_snr corresponds to the SNR we would like for the frames files, which is 
    achieved by adjusting the distance. 
    """
    time, h_plus, h_cross, amp = EvaluateSurrogate(sur, params_dict)
    timeH, strainH, timeL, strainL, timeV, strainV = PadAndProject(time, h_plus, h_cross, params_dict)
    
    ## Update distance for target SNR
    strainH, strainL, params_dict = TargetSNR(timeH, strainH, timeL, strainL, timeV, strainV, params_dict, desired_snr)
    GenerateFrames(p_out, timeH, strainH, timeL, strainL, timeV, strainV, params_dict)
    
    ## Compute SNRs
    SNR_H = ComputeSNR(timeH, strainH)
    SNR_L = ComputeSNR(timeL, strainL)
    SNR_V = ComputeSNR(timeV, strainV)

    ## Print out the parameter values
    params_dict = FullParamsDictionary(params_dict)
    params_dict['H_SNR'] = SNR_H
    params_dict['L_SNR'] = SNR_L
    params_dict['V_SNR'] = SNR_V
    with open('BilbyPE/' + p_out + '/parameters.json', 'w') as fp:
        json.dump(params_dict, fp)
    fp.close()


### Actually generate frames files

In [5]:
# ## Parameters under consideration for this study
My_Params_Dict = {'dt' : 1/2048, 't_gps' : 1126259462.0, 'peak_time_in_segment' : 8.0, 'segment_length' : 16.0, \
               'dist_mpc' : 400.0, 'mass' : 68.0, 'q' : 1.2212532137858916, \
               'a_1' : [0, 0, 0.329892752405], 'a_2' : [0, 0, -0.439940981499], \
               'f_low' : 0, 'theta' : pi, 'phi' : 0, \
               'ra' : 1.952318922, 'dec' : -1.26967171703, 'pol' : 0.824043851821}
 

for SNR in [100]:
    for ell in ['0p0']:
        print('--------------------------------------------')
        EvaluateNRWaveform('Waveforms/Lev5/dCS_Strain_' + ell + '.h5', 'PHENOM_dCS_' + ell + '_' + str(SNR), My_Params_Dict, SNR, ell=ell)

--------------------------------------------
t0 parameter:  1126259462.0


Time delay between detectors -0.006985699700979881 -0.004281157268499984
geocenter time delay H:  0.014694082894586626
geocenter time delay L:  0.007708383193606742
geocenter time delay V:  0.010412925626086639
Peak time in H initially: 0.00046890974044799805
Peak time in H updated: 1126259462.0
Peak time in L updated: 1126259461.989565
Peak time in V updated: 1126259461.9926715
Segment start and end:  1126259454.0 1126259470.0
t0 is in the time array:  True
Peak time in H after interpolation: 1126259462.0
Peak time in L after interpolation: 1126259461.989746
Peak time in V after interpolation: 1126259461.9926758
Current SNR  112.69448184926898
Current distance 400.0
Updated distance 450.7779273970759
Network SNR:  100.0


### Plot the frame files

In [ ]:
def PlotInjectedWaveforms():
    
    """ Produce a plot of injected waveforms read from frames files """

    fig, ax = plt.subplots(figsize=(10, 6))

    base = '/home/maria.okounkova/BeyondGRAnalysis/BilbyPE/'

    ## Get injected paramters
    p = base + 'PHENOM_dCS_0p0_75'
    params = GetInjectedParameters(p)
    print(params)
    
    for det in ["H1", "L1", "V1"]:
        strain = TimeSeries.read(source = p + '/Frames/' + det + '.gwf', channel=det+":LDAS_STRAIN")
        time = np.array(strain.times)
        plt.plot(time, strain, '-', lw=1.5, label = det)

    t_gps = 1126259462.0
    plt.xlim(t_gps - 1.0, t_gps + 0.2)
    plt.xlabel('Time [sec]')
    plt.ylabel(r'$h(t)$')
    legend = plt.legend(fontsize=24, loc = 'lower left', ncol=2, frameon=False)
    plt.grid()
    plt.tight_layout()
    #plt.savefig('InjectedWaveforms.pdf')
    plt.show()
    
#PlotInjectedWaveforms()